In [8]:
import numpy as np

In [11]:
A = 0.005 #area of the element
k = 1.4 #thermal conductivity
K_tri = np.array([[1,-0.5,0,-0.5,0,0,0,0,0],
         [-0.5,2,-0.5,0,-1,0,0,0,0],
         [0,-0.5,1,0,0,-0.5,0,0,0],
         [-0.5,0,0,2,-1,0,-1/2,0,0],
         [0,-1,0,-1,4,-1,0,-1,0],
         [0,0,-0.5,0,-1,2,0,0,-0.5],
         [0,0,0,-0.5,0,0,1,-0.5,0],
         [0,0,0,0,-1,0,-0.5,2,-0.5],
         [0,0,0,0,-0.5,-0.5,0,-0.5,1]])*k/8/A
print(K_tri)

[[ 35.  -17.5   0.  -17.5   0.    0.    0.    0.    0. ]
 [-17.5  70.  -17.5   0.  -35.    0.    0.    0.    0. ]
 [  0.  -17.5  35.    0.    0.  -17.5   0.    0.    0. ]
 [-17.5   0.    0.   70.  -35.    0.  -17.5   0.    0. ]
 [  0.  -35.    0.  -35.  140.  -35.    0.  -35.    0. ]
 [  0.    0.  -17.5   0.  -35.   70.    0.    0.  -17.5]
 [  0.    0.    0.  -17.5   0.    0.   35.  -17.5   0. ]
 [  0.    0.    0.    0.  -35.    0.  -17.5  70.  -17.5]
 [  0.    0.    0.    0.  -17.5 -17.5   0.  -17.5  35. ]]


In [12]:
#initial condition
T_right = 300
T_left = 280

In [17]:
F = np.zeros((9,1))
F[0] = 280
K_tri[0,:] = 0
K_tri[0,0] = 1
F[3] = 280
K_tri[3,:] = 0
K_tri[3,3] = 1
F[6] = 280
K_tri[6,:] = 0
K_tri[6,6] = 1
F[2] = 300
K_tri[2,:] = 0
K_tri[2,2] = 1
F[5] = 300
K_tri[5,:] = 0
K_tri[5,5] = 1
F[8] = 300
K_tri[8,:] = 0
K_tri[8,8] = 1
K_tri

array([[  1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [-17.5,  70. , -17.5,   0. , -35. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. , -35. ,   0. , -35. , 140. , -35. ,   0. , -35. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   1. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   1. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. , -35. ,   0. , -17.5,  70. , -17.5],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   1. ]])

In [18]:
U = np.dot(np.linalg.pinv(K_tri), F)
U

array([[280.],
       [290.],
       [300.],
       [280.],
       [290.],
       [300.],
       [280.],
       [290.],
       [300.]])

In [1]:
#matrix K using triangular finite elements
def tri_subK_matrix(A, b1, b2, b3, c1, c2, c3):
    A_14 = 1/4/A
    K11 = A_14*(b1**2+c1**2)
    K12 = A_14*(b1*b2+c1*c2)
    K13 = A_14*(b1*b3+c1*c3)
    K23 = A_14*(b2*b3+c2*c3)
    K22 = A_14*(b2**2+c2**2)
    K33 = A_14*(b3**2+c3**2)
    return [K11,K12,K13,K23,K22,K33]

def tri_K_matrix(A, b1, b2, b3, c1, c2, c3):
    A_14 = 1/4/A
    K11 = A_14*(b1**2+c1**2)
    K12 = A_14*(b1*b2+c1*c2)
    K13 = A_14*(b1*b3+c1*c3)
    K23 = A_14*(b2*b3+c2*c3)
    K22 = A_14*(b2**2+c2**2)
    K33 = A_14*(b3**2+c3**2)
    return [K11,K12,K13,K23,K22,K33]

In [2]:
hx = hy = 0.1
#matrix K using rectangular finite elements
def rec_K_matrix(hx, hy):
    K11 = (hx**2+hy**2)/hx/hy/3 # K22,K33,K44
    K12 = (hx**2-2*hy**2)/hx/hy/6 # K21,K34,K43
    K13 = -(hx**2+hy**2)/hx/hy/6 # K31,K24,K42
    K14 = -(2*hx**2-hy**2)/hx/hy/6 # K41,K23,K32
    return K11,K12,K13,K14

#insert correct index into final K matrix
def fill_index(a,b,c,d):
    K_tri = np.zeros((9,9))
    A,B,C,D = rec_K_matrix(hx,hy)
    K_tri[a,a] = K_tri[b,b] = K_tri[c,c] = K_tri[d,d] = A
    K_tri[a,b] = K_tri[b,a] = K_tri[c,d] = K_tri[d,c] = B
    K_tri[a,c] = K_tri[c,a] = K_tri[b,d] = K_tri[d,b] = C
    K_tri[a,d] = K_tri[d,a] = K_tri[b,c] = K_tri[c,b] = D
    return K_tri

In [25]:
K_rec = k*(fill_index(0,1,4,3) +  fill_index(1,2,5,4) +  fill_index(3,4,7,6) +  fill_index(4,5,8,7))
F = np.zeros((9,1))
F[0] = 280
K_rec[0,:] = 0
K_rec[0,0] = 1
F[3] = 280
K_rec[3,:] = 0
K_rec[3,3] = 1
F[6] = 280
K_rec[6,:] = 0
K_rec[6,6] = 1
F[2] = 300
K_rec[2,:] = 0
K_rec[2,2] = 1
F[5] = 300
K_rec[5,:] = 0
K_rec[5,5] = 1
F[8] = 300
K_rec[8,:] = 0
K_rec[8,8] = 1
K_rec

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.23333333,  1.86666667, -0.23333333, -0.46666667, -0.46666667,
        -0.46666667,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.46666667, -0.46666667, -0.46666667, -0.46666667,  3.73333333,
        -0.46666667, -0.46666667, -0.46666667, -0.46666667],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.46666667, -0

In [26]:
U_rec = np.dot(np.linalg.pinv(K_rec), F)
U_rec

array([[280.],
       [290.],
       [300.],
       [280.],
       [290.],
       [300.],
       [280.],
       [290.],
       [300.]])